# Step 2. Prediction target protein structure with ESMFold


The example uses malic enzyme from Trypanosoma cruzi, which is the cause of Chagas disease. Ligand (PDB ID SZD) is one of the inhibitors in Scheme 1 in [this paper](https://pubs.acs.org/doi/epdf/10.1021/acsinfecdis.1c00231)

Even though we do have the crystal structure of the protein-ligand complex (PDB ID 6W59), we will still use ESMFold to predict the protein structure for demo purpose. 

The sequence of the protein is as follows:

```bash
>6W59_SZD_protein_A
AILTDRYINRGTAFTMEERQKLHILGRLPPVVETLEEQVARVYGQVKKYEKPINRYQHLVSVHSTNTTLY
YATILAHLEEMLPIIYTPTVGEACMEYSHLFFRERGVYFNRLYKGQFRNIMRDAGYQKVEVVVITDGSRI
LGLGDLGSNGIGISIGKCSLYVAGAGIDPRLIVPVILDVGTNNERYLQDKDYLGMREKRLGDEEFYELLD
EFMEAASAEWPNAVIQFEDFSNNHCFDIMERYQKKYRCFNDDIQGTGAVIAAGFLNAIKLSGVSPLQQRI
VVFGAGSAAVGVANNIAALAARMYKFPVQDLVKTFYLVDTKGLVTTTRGDQLAAHKKLLARTDVSAEDSA
KLRTLEEIVRFVKPTTLLGLGGVGPAFTEEIVKMVMQNTERPIIFPLSNPTSKAEVTPENAYKWTNGAAI
VASGSPFPPTTIGGKTFKPSQGNNLYVFPGVGLGCALAQPTHIPEELLLTASESLNLLTTEGDLREGRLY
PPLEDIHNISANVATDVILEAQRMKIDNNKKLPRTRDELLAFVKKAMWKPVYSG
```


## Set up ESMFold NIM

### Done by download.sh
1. Open a Terminal.
2. Run `docker login --username '$oauthtoken' --password $NGC_CLI_API_KEY nvcr.io`
3. Run `docker pull nvcr.io/nvidia/nim/bionemo_esmfold_nim:24.03.01`
4. Run `ngc config set`
   - API key: enter NGC API key.
   - CLI output: accept default (ascii) by pressing Enter 
   - org: Choose from the list the NGC org that starts with "DA" (this is the org for downloadable NIM)
   - team: Whichever is fine
   - ace: Choose "no-ace"
4. Run the following commands to download the ESMFold NIM and weights.

```bash
    mkdir -p ~/esmfold-nim/{weights,models}
    ngc registry model download-version nvidia/nim/bionemo-esmfold:protein-folding_noarchx1_bf16_24.03 --dest esmfold-nim/models/
    curl -L https://dl.fbaipublicfiles.com/fair-esm/models/esmfold_3B_v1.pt --output esmfold-nim/weights/esmfold_3B_v1.pt
    curl -L https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t36_3B_UR50D.pt --output esmfold-nim/weights/esm2_t36_3B_UR50D.pt
    curl -L https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t36_3B_UR50D-contact-regression.pt --output esmfold-nim/weights/esm2_t36_3B_UR50D-contact-regression.pt
```

### Start here
5. Run the following to start the ESMFold server

```bash
   docker run --rm -it \
   --name bionemo-esmfold \
   --gpus all \
   -v $(pwd)/esmfold-nim/models/bionemo-esmfold_vprotein-folding_noarchx1_bf16_24.03/:/config/models \
   -v $(pwd)/esmfold-nim/weights/:/esm_models \
   -e MODEL_PATH=/config/models \
   -p 8008:8008 \
   nvcr.io/nvidia/nim/bionemo_esmfold_nim:24.03.01
 ```

6. You may return to this notebook and start the following cells once you see a response like this on your terminal:

 ```bash
    I0421 21:04:17.461556 81 grpc_server.cc:2519] Started GRPCInferenceService at 0.0.0.0:8001
    I0421 21:04:17.461909 81 http_server.cc:4637] Started HTTPService at 0.0.0.0:8000
    I0421 21:04:17.503485 81 http_server.cc:320] Started Metrics Service at 0.0.0.0:8002
 ```


## Intiate & Health Check

In [22]:
from scripts.nim import ESMFoldNIM

# Update to IP address of the EC2 instance
base_url = "http://0.0.0.0:8008"

# initiate ESMFoldNIM object
esmfold_nim = ESMFoldNIM(base_url)
print("Health check url: ", esmfold_nim.health_check_url)
print("Query url: ", esmfold_nim.query_url)

Health check url:  http://0.0.0.0:8008/health/ready
Query url:  http://0.0.0.0:8008/protein-structure/esmfold/predict


In [23]:
# run health check, will return True if the health check passed, False otherwise
esmfold_nim.check_health()

Health check passed


True

## Predict Structure

In [24]:
# sequence of the protein
sequence = "AILTDRYINRGTAFTMEERQKLHILGRLPPVVETLEEQVARVYGQVKKYEKPINRYQHLVSVHSTNTTLYYATILAHLEEMLPIIYTPTVGEACMEYSHLFFRERGVYFNRLYKGQFRNIMRDAGYQKVEVVVITDGSRILGLGDLGSNGIGISIGKCSLYVAGAGIDPRLIVPVILDVGTNNERYLQDKDYLGMREKRLGDEEFYELLDEFMEAASAEWPNAVIQFEDFSNNHCFDIMERYQKKYRCFNDDIQGTGAVIAAGFLNAIKLSGVSPLQQRIVVFGAGSAAVGVANNIAALAARMYKFPVQDLVKTFYLVDTKGLVTTTRGDQLAAHKKLLARTDVSAEDSAKLRTLEEIVRFVKPTTLLGLGGVGPAFTEEIVKMVMQNTERPIIFPLSNPTSKAEVTPENAYKWTNGAAIVASGSPFPPTTIGGKTFKPSQGNNLYVFPGVGLGCALAQPTHIPEELLLTASESLNLLTTEGDLREGRLYPPLEDIHNISANVATDVILEAQRMKIDNNKKLPRTRDELLAFVKKAMWKPVYSG"

print(f"length of protein: {len(sequence)} aa")

length of protein: 544 aa


In [25]:
%%time
result = esmfold_nim.predict(sequence)

Request successful
CPU times: user 4.38 ms, sys: 63 μs, total: 4.44 ms
Wall time: 21.4 s


## Analysis & Visualization

In [26]:
result.keys()

dict_keys(['pdbs'])

In [27]:
# only 1 sequence is allowed each time
assert len(result["pdbs"]) == 1

# result['pdbs'][0] is the str representation of the PDB file

In [28]:
import os
from scripts.utils import prepare_output_directory


# set up the output directory
output_dir = "output/esmfold_result"
prepare_output_directory(output_dir)

# Write PDB file
fp = os.path.join(output_dir, f"predicted_protein.pdb")
with open(fp, "w") as f:
    f.write(result["pdbs"][0])

We can take a look at the predicted protein structure and its pLDDT score. The score is stored as B-factor in the returned results. 

To align with the [EBI/AlphaFold database](https://alphafold.ebi.ac.uk/), we will use the following colors for the pLDDT score

![plddt](https://res.cloudinary.com/dpfqlyh21/image/upload/v1705026011/obsidian/izrfmiepbzpnzm2aoqwh.png)

We display the predicted structure with these colors using py3Dmol.


In [2]:
from scripts.visualization import load_protein, align_protein

In [3]:
view = load_protein(
    pdb_file_path = 'output/esmfold_result/predicted_protein.pdb', 
    width=800, 
    height=500
)
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

To show how the predicted structure aligns with the crystal structure, we will use the crystal structure from (PDB ID 6W59) and calcualte the backbone RMSD: 

In [4]:
view = align_protein(
    pred_pdb_file = 'output/esmfold_result/predicted_protein.pdb', # prediction
    true_pdb_file = 'data/6W59_SZD_protein.pdb', # true crystal structure
    output_dir = 'output/esmfold_result', # output directory to store the aligned poses so that py3dmol can load back
    pred_color = 'cyan', # color for the predicted structure
    true_color = 'green', # color for the true structure
    width=800, 
    height=500
)
view.show()

 Match: read scoring matrix.
 Match: assigning 544 x 544 pairwise scores.
 MatchAlign: aligning residues (544 vs 544)...
 MatchAlign: score 2802.000
 ExecutiveAlign: 2176 atoms aligned.
 ExecutiveRMS: 114 atoms rejected during cycle 1 (RMSD=2.15).
 ExecutiveRMS: 93 atoms rejected during cycle 2 (RMSD=1.83).
 ExecutiveRMS: 66 atoms rejected during cycle 3 (RMSD=1.65).
 ExecutiveRMS: 61 atoms rejected during cycle 4 (RMSD=1.54).
 ExecutiveRMS: 30 atoms rejected during cycle 5 (RMSD=1.45).
 Executive: RMSD =    1.405 (1812 to 1812 atoms)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

To show how the predicted structure aligns with the crystal structure, we will use the crystal structure from (PDB ID 6W59) and calculate the backbone RMSD.

You can see that the backbone RMSD is roughly about 1.4 A. According to the [ESMFold paper](https://www.science.org/doi/10.1126/science.ade2574), on the CAMEO test set, ESMFold predictions
have a median all-atom RMSD95 (root-mean-squared deviation at 95% residue coverage) of 1.91 A and backbone RMSD95 of 1.33 A. Our result is similar with the reported median number.  
 
Note that the ESMFold prediction does not differentiate between holo/apo structures.

## Prepare receptor structure for docking

In this example, our oraclce is composed of 2 docking models: Diffdock and Unidock. 

Diffdock takes a PDB file as input, but other docking models (e.g Unidock) might require a PDBQT. To prepare for docking, we will use the [reduce library](https://github.com/rlabduke/reduce/tree/master) to clean the protein structures, which includes adding hydrogens to a PDB file,  standardizing the bond lengths of existing hydrogens, and optimization of adjustable groups (OH, SH, NH3+, Met-CH3, and Asn, Gln and His sidechain orientation), etc.  Note that ESMFold output files do not include hydrogens, therefore this step is necessary. 

### Make a folder for input files for Diffdock/Unidock

In [11]:
from scripts.utils import prepare_output_directory

prepare_output_directory('output/docking_input')

### Clean structure

- Input: predicted protein structure in PDB format
- Output: cleaned protein structure in PDB format ready for Diffdock

In [12]:
!reduce -build output/esmfold_result/predicted_protein.pdb > output/docking_input/protein.pdb

Processing file: "output/esmfold_result/predicted_protein.pdb"
Building His ring NH Hydrogens.
Building or keeping OH & SH Hydrogens.
Rotating existing OH & SH Hydrogens
ERROR CTab(/usr/local/reduce_wwPDB_het_dict.txt): could not open
SKIPPED H( A 247 ARGHH21 ): A 247 ARG NH2  bonds- A 221 PRO O    (H bumps)
SKIPPED H( A 247 ARGHH22 ): A 247 ARG NH2  bonds- A 221 PRO O    (H bumps)
VDW dot density = 16/A^2
Probe radius = 0.25A
Orientation penalty scale = 1 (100%)
Eliminate contacts within 3 bonds.
Ignore atoms with |occupancy| <= 0.01 during adjustments.
Waters ignored if B-Factor >= 40 or |occupancy| < 0.66
Aromatic rings in amino acids accept hydrogen bonds.
Flipping Asn, Gln and His groups.
For each flip state, bumps where gap is more than 0.4A are indicated with '!'.
Rotating NH3 Hydrogens.
Not processing Met methyls.
 Singles(size 130): A   7 TYR OH  : A   9 ASN     : A  12 THR OG1 : A  15 THR OG1 
  : A  20 GLN     : A  21 LYS NZ  : A  23 HIS     : A  34 THR OG1 : A  43 TYR OH  


### Convert to PDBQT

This step is only needed if the the docking model (such as Unidock) requires a PDBQT file. 

- Input: cleaned protein structure in PDB format
- Output: cleaned protein structure in PDBQT format

In [1]:
!obabel -ipdb output/docking_input/protein.pdb -xr -xh -opdbqt > output/docking_input/protein.pdbqt

1 molecule converted
